# [Getting Started with Diffusers for Text-to-Image](https://pyimagesearch.com/2024/01/22/getting-started-with-diffusers-for-text-to-image/)

Using Diffusers library from Hugging Face, we'll generate images from text.

Diffusion Models are generative models, meaning that they are used to generate data similar to the data on which they are trained. These models work by destroying training data through the successive addition of Gaussian noise and then learning to recover the data by reversing this noising process.

Diffusion probabilistic models are parameterized Markov chains trained to gradually denoise data. 

<!--  -->